In [2]:
import time
import serial
import struct
import matplotlib.pyplot as plt
import IPython

In [210]:
port = serial.Serial("COM4", 115200, timeout=1)

In [217]:
buf = b"OUT"+struct.pack("<hhhhhhhh", 1100,1200,300,400,500,600,700,800)
#buf = b"OUT"+struct.pack("<h", 1500)*8
buf = bytes([len(buf)+1])+buf+b"\x00"
buf

b'\x14OUTL\x04\xb0\x04,\x01\x90\x01\xf4\x01X\x02\xbc\x02 \x03\x00'

In [218]:
port.write(buf)

21

In [219]:
print(port.read(4096).decode())

20 79 85 84 76 4 176 4 44 1 144 1 244 1 88 2 188 2 32 3 0 
79 85 84 76 4 176 4 44 1 144 1 244 1 88 2 188 2 32 3 
1100 1200 300 400 500 600 700 800 



In [220]:
port.close()

In [50]:
def cobs_decode(buf):
    out = bytearray()
    if len(buf) == 0:
        return out
    in_ptr = 0
    while in_ptr < (len(buf) - 1):
        out += b'\x00'
        frame_len = buf[in_ptr]
        out += buf[in_ptr+1:][:frame_len-1]
        in_ptr += frame_len
    if buf[0] != 1:
        out = out[1:]
    return out

In [51]:
def get_packet():
    buf = port.read_until(b'\x00')
    buf = cobs_decode(buf)
    return buf

In [36]:
def get_imu_reading():
    buf = port.read_until(b'\x00')
    buf = cobs_decode(buf)
    if len(buf) != 12:
        return None
    imur = struct.unpack("<hhhhhh", buf)
    return imur

In [53]:
def measure_rate(duration=5):
    port.read(4096)
    start = time.monotonic()
    count = 0
    while time.monotonic() - start < duration:
        imur = get_imu_reading()
        count += 1
    return count / duration

measure_rate()

280.2

In [89]:
def loop():
    port.read(4096)
    count = 10
    while count > 0:
        print(get_packet())
        count -= 1
        
loop()

bytearray(b'IMU\xb4\x02\xd0\xef\x8c4\xbf\xff\xfb\xff\x07\x00')
bytearray(b'PPM\x01\x02\xd0\xef\x8c\xd6\x05\xd3\x05{\x03\xd2\x05\t\x06\xd2\x05')
bytearray(b'IMU\x98\x02\xd0\xef\x864\xbd\xff\xfd\xff\x08\x00')
bytearray(b'PPM\x01\x02\xd0\xef\x86\xd6\x05\xd3\x05{\x03\xd2\x05\t\x06\xd1\x05')
bytearray(b'IMU\x9a\x02\xcc\xefv4\xbd\xff\xfb\xff\x06\x00')
bytearray(b'PPM\x01\x02\xcc\xefv\xd6\x05\xd3\x05{\x03\xd2\x05\t\x06\xd2\x05')
bytearray(b'IMU\x9c\x02\xb0\xeff4\xbd\xff\xfe\xff\x07\x00')
bytearray(b'PPM\x01\x02\xb0\xeff\xd6\x05\xd3\x05{\x03\xd2\x05\t\x06\xd2\x05')
bytearray(b'IMU\xbc\x02\xc4\xef\xa04\xbe\xff\xfb\xff\x07\x00')
bytearray(b'PPM\x01\x02\xc4\xef\xa0\xd6\x05\xd3\x05{\x03\xd2\x05\t\x06\xd2\x05')
